# Booz Allen Spring 2025 Codefest: Challenge 1 
### Isaiah Byrd, Kyler Gelissen, David Ameh

In [13]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")
#print(path)

In [14]:
import os

# Create a directory to store the dataset
dataset_dir = "garbage_classification_dataset"
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
    print(f"Directory {dataset_dir} created.")
else:
    pass
    print(f"Directory {dataset_dir} already exists.")

Directory garbage_classification_dataset already exists.


In [ ]:
import subprocess
import sys

# Function to install a package
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = [
    "matplotlib",
    "pandas",
    "numpy",
    "torch",
    "torchvision",
    "opencv-python",
    "kagglehub"
]

# Install each package if not already installed
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        install(package)

# Importing the libraries
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import torch as th
import cv2
import json
import random
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Dataset

In [16]:
#Set divice to GPU if available
divice = th.device("cuda" if th.cuda.is_available() else "cpu")

#Change the memory fraction to limit the GPU memory usage
#Set the memory fraction to 60% of the total GPU memory
memory_fraction = 0.80

#Limits the GPU memory usage to 60% 
if(divice.type == "cuda"):
    print("GPU is available")
    th.cuda.set_memory_fraction(memory_fraction, divice=divice.index)

In [17]:
#Defines the data transforms
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)), #Image needs to be resized to 224x224 for ResNet requirement
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #Standard normalization for ResNet
])

#Load the dataset
complete_dataset = datasets.ImageFolder(root=path, transform=data_transforms) 
dataset_size = len(complete_dataset)

In [18]:
#We are going to do a 80-10-10 split of the dataset to start
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size
print(f"Dataset size: {dataset_size}")
print(f"Train size: {train_size}")
print(f"Validation size: {val_size}")
print(f"Test size: {test_size}")

#Using random_split to split the dataset into train, validation and test sets
train_dataset, val_dataset, test_dataset = th.utils.data.random_split(complete_dataset, [train_size, val_size, test_size])

Dataset size: 19762
Train size: 15809
Validation size: 1976
Test size: 1977


In [19]:
#Creating the dataloaders
batch_size = 16 #Update this if memory permits
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Manually defining the class names as per the dataset
class_names = ['Metal', 'Glass', 'Biological', 'Paper', 'Battery', 'Trash', 'Cardboard', 'Shoes', 'Clothes', 'Plastic']
num_classes = len(class_names)

In [20]:
model = models.resnet50(pretrained=True) #Using ResNet50 as the base model for transfer learning

#This takes the last layer of the model and replaces it with a new one
num_ftrs = model.fc.in_features 
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(divice) #Moving the model to the device

In [21]:
criterion = nn.CrossEntropyLoss()
#Learning rate is set to 1e-4 as a starting point, update this if needed
learning_rate = 1.e-4 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
num_empochs = 2 #Update this if needed
train_accuracy_history = []
val_accuracy_history = []

In [23]:
#Main training loop
for epoch in  range(num_empochs):
    model.train() #Sets the model to training mode
    running_corrects = 0
    running_samples = 0

    for inputs, labels in train_loader:
        #Move the inputs and labels to the device
        inputs = inputs.to(divice)
        labels = labels.to(divice)

        #Zero the parameter gradients
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward() #Backpropagation step
        optimizer.step() #Optimizer step

        #Get the stupid predictions
        preds = th.argmax(outputs, dim=1)
        preds = preds.cpu().numpy()
        labels = labels.cpu().numpy()
        running_corrects += np.sum(preds == labels)
        running_samples += labels.shape[0]
    
    #Calculate the accuracy
    epoch_train_accuracy = running_corrects / running_samples
    train_accuracy_history.append(epoch_train_accuracy)

    #Validation step
    model.eval() #Sets the model to evaluation mode
    running_corrects = 0
    running_samples = 0
    with th.no_grad(): #Disables gradient calculation
        for inputs, labels in val_loader:
            inputs = inputs.to(divice)
            labels = labels.to(divice)
            outputs = model(inputs)

            preds = th.argmax(outputs, dim=1)
            preds = preds.cpu().numpy()
            labels = labels.cpu().numpy()
            running_corrects += np.sum(preds == labels)
            running_samples += labels.shape[0]

    #Calculate the accuracy
    epoch_val_accuracy = running_corrects / running_samples
    val_accuracy_history.append(epoch_val_accuracy)

    print(f"Epoch {epoch+1}/{num_empochs} - Train accuracy: {epoch_train_accuracy:.4f} - Validation accuracy: {epoch_val_accuracy:.4f}")

train_accuracy_history = np.array(train_accuracy_history)
val_accuracy_history = np.array(val_accuracy_history)

KeyboardInterrupt: 